In [34]:
import numpy as np
import pandas.api.types
import re # Import regex module


In [40]:
import pandas as pd

excel_file = pd.ExcelFile('data/EXP_3.xlsx')
sheet_names = excel_file.sheet_names

print("Sheet names in 'EXP_3.xlsx':")
for sheet in sheet_names:
    print(f"- {sheet}")

Sheet names in 'EXP_3.xlsx':
- Seedlings measurements 1952024
- Water quality measurments Senso
- Water quality parametersPortabl
- Nutrients Water Consumptions
- Harvest measurements 1862024


In [35]:
import pandas as pd

loaded_cleaned_dfs = {}

for sheet_name in sheet_names:
    cleaned_sheet_name = sheet_name.replace(' ', '_').replace('/', '_').replace('\\', '_').replace(':', '_')
    csv_filename = f"cleaned_{cleaned_sheet_name}.csv"
    try:
        df = pd.read_csv(csv_filename)
        loaded_cleaned_dfs[sheet_name] = df
        print(f"Successfully loaded '{csv_filename}' into a DataFrame.")
        print(f"Shape of '{sheet_name}': {df.shape}")
    except FileNotFoundError:
        print(f"Warning: '{csv_filename}' not found. It might have been empty after cleaning.")
    except pd.errors.EmptyDataError:
        print(f"Warning: '{csv_filename}' is empty and could not be loaded into a DataFrame.")

print("\nAll available cleaned CSV files have been loaded.")


All available cleaned CSV files have been loaded.


In [36]:
# Helper function to load and preprocess headers
def _load_and_preprocess_headers(sheet_name, excel_file_path='data/EXP_3.xlsx'):
    df = None
    if sheet_name in ['Water quality parametersPortabl', 'Nutrients Water Consumptions', 'Water quality measurments Senso']:
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name, header=[0, 1])
        new_columns = []
        for col_tuple in df.columns:
            top_level = str(col_tuple[0]).strip() if not pd.isna(col_tuple[0]) else ''
            second_level = str(col_tuple[1]).strip() if not pd.isna(col_tuple[1]) else ''

            if top_level.startswith('Unnamed:') and second_level:
                new_columns.append(second_level)
            elif top_level and second_level and top_level != second_level:
                new_columns.append(f"{top_level}_{second_level}")
            elif top_level:
                new_columns.append(top_level)
            elif second_level:
                new_columns.append(second_level)
            else:
                new_columns.append(f"Unnamed_Col_{len(new_columns)}")
        df.columns = new_columns
        if df.shape[0] > 0 and df.iloc[0].isnull().mean() > 0.5:
            print(f"  - Detected largely empty first row after header processing for {sheet_name}; dropping it.")
            df = df.iloc[1:].copy()
            df.reset_index(drop=True, inplace=True)
    else:
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name, header=1)
        if any('Unnamed' in str(col) for col in df.columns):
            print(f"  - Detected 'Unnamed' columns after header=1; promoting df.iloc[0] to header for {sheet_name}.")
            new_header = df.iloc[0]
            df = df[1:]
            df.columns = new_header

    df.columns = [col.replace(' ', '_').replace('/', '_').replace('\\', '_').replace('.', '').strip() for col in df.columns]
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i, i_val in enumerate(cols[cols == dup].index.values.tolist())]
    df.columns = cols
    return df

In [24]:
# Helper function for initial inspection
def _initial_inspection(df):
    print("\n1. Initial Inspection (Head):")
    print(df.head())
    print("\n2. Initial Inspection (Info):")
    df.info()
    print("\n3. Initial Inspection (Descriptive Statistics):")
    print(df.describe(include='all'))

In [25]:
# Helper function to drop entirely empty rows
def _drop_entirely_empty_rows(df, sheet_name):
    initial_rows = df.shape[0]
    df.dropna(how='all', inplace=True)
    rows_dropped = initial_rows - df.shape[0]
    if rows_dropped > 0:
        print(f"  - Dropped {rows_dropped} rows where all values were missing from {sheet_name}.")
    return df

In [26]:
# Helper function to handle missing values (fill numerical with 0)
def _handle_missing_values(df):
    print("\n4. Handling Missing Values:")
    missing_before = df.isnull().sum().sum()
    if missing_before > 0:
        print(f"Missing values before filling: {missing_before}")
        numerical_cols = df.select_dtypes(include=np.number).columns
        if not numerical_cols.empty:
            df[numerical_cols] = df[numerical_cols].fillna(0)
            print(f"  - Filled missing numerical values with 0.")
        missing_after_fillna = df.isnull().sum().sum()
        if missing_after_fillna > 0:
            print(f"  - Remaining missing values (non-numerical) after filling numerical columns with 0: {missing_after_fillna}")
        else:
            print(f"  - No remaining missing values after filling numerical columns with 0.")
    else:
        print("No missing values found.")
    return df

In [27]:
# Helper function to handle duplicates
def _handle_duplicates(df):
    print("\n5. Handling Duplicates:")
    duplicates_before = df.duplicated().sum()
    if duplicates_before > 0:
        print(f"Duplicate rows before dropping: {duplicates_before}")
        df.drop_duplicates(inplace=True)
        duplicates_after = df.duplicated().sum()
        print(f"Duplicate rows after dropping: {duplicates_after}")
    else:
        print("No duplicate rows found.")
    return df

In [39]:
# Helper function to standardize data types
def _standardize_datatypes(df):
    print("\n6. Standardizing Data Types:")
    for col in df.columns:
        if df[col].dtype == 'object':
            if df[col].astype(str).str.contains(r'^\d+\.?\d*\*\d+\.?\d*$', regex=True).any():
                print(f"  - Detected 'NUMBER*NUMBER' pattern in column '{col}'. Attempting multiplication.")
                def multiply_pattern(value):
                    if isinstance(value, str) and re.fullmatch(r'^\d+\.?\d*\*\d+\.?\d*$', value):
                        try:
                            num1_str, num2_str = value.split('*')
                            return float(num1_str) * float(num2_str)
                        except ValueError:
                            return np.nan
                    return value
                df[col] = df[col].apply(multiply_pattern)
                df[col] = pd.to_numeric(df[col], errors='coerce')
                if pandas.api.types.is_numeric_dtype(df[col]):
                    print(f"  - Multiplied and converted column '{col}' to numeric.")
                    continue
                else:
                    print(f"  - Could not fully convert column '{col}' to numeric after multiplication.")

            if 'date' in str(col).lower():
                converted_date_series = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
                if not converted_date_series.isnull().all():
                    df[col] = converted_date_series
                    print(f"  - Converted column '{col}' to datetime.")
                    if df[col].isnull().any():
                        print(f"    Warning: Some values in '{col}' could not be converted to datetime and became NaT.")
                    continue
                else:
                    print(f"  - Column '{col}' (date-like name) could not be effectively converted to datetime; values might be non-date strings or all NaNs.")

            converted_numeric_series = pd.to_numeric(df[col], errors='coerce')
            if pandas.api.types.is_numeric_dtype(converted_numeric_series) and not converted_numeric_series.isnull().all():
                df[col] = converted_numeric_series
                print(f"  - Converted column '{col}' to numeric.")
                continue

            if df[col].dtype == 'object':
                if df[col].nunique() / len(df[col]) < 0.5:
                    df[col] = df[col].astype('category')
                    print(f"  - Converted column '{col}' to category.")
                else:
                    print(f"  - Column '{col}' remains as object (too many unique values for category conversion or could not be converted to numeric/datetime). ")
    df.info()
    return df

In [29]:
# Helper function to address categorical inconsistencies
def _address_categorical_inconsistencies(df):
    print("\n7. Addressing Inconsistencies (Categorical Columns):")
    for col in df.select_dtypes(include='category').columns:
        print(f"  - Unique values for '{col}': {df[col].unique()}")
        df[col] = df[col].astype(str).str.strip().str.lower().astype('category')
        print(f"    Standardized unique values for '{col}': {df[col].unique()}")
    return df

In [30]:
# Helper function for outlier inspection
def _outlier_inspection(df):
    print("\n8. Outlier Inspection (Numerical Columns):")
    for col in df.select_dtypes(include=np.number).columns:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
        if outliers_count > 0:
            print(f"  - Column '{col}': {outliers_count} potential outliers detected (outside 1.5*IQR).")
        else:
            print(f"  - Column '{col}': No obvious outliers detected (within 1.5*IQR).")
    return df

In [31]:
# Helper function to format numerical columns
def _format_numerical_columns(df):
    print("\n9. Formatting numerical columns to 2 decimal places:")
    for col in df.select_dtypes(include=np.number).columns:
        df[col] = df[col].round(2)
        print(f"  - Rounded column '{col}' to 2 decimal places.")
    return df

In [37]:
# Helper function to save cleaned sheet
def _save_cleaned_sheet(df, sheet_name):
    sheet_name_to_filename_map = {
        'Seedlings measurements 1952024': 'Seedlings_V3_clean',
        'Water quality measurments Senso': 'WaterQualityMeasurments_V3_clean',
        'Water quality parametersPortabl': 'WaterQualityParametersPortable_V3_clean',
        'Nutrients Water Consumptions': 'Nutrients_V3_clean',
        'Harvest measurements 1862024': 'Harvest_V3_clean'
    }

    base_cleaned_name = sheet_name_to_filename_map.get(
        sheet_name,
        sheet_name.replace(' ', '_').replace('/', '_').replace('\\', '_').replace(':', '_') + '_cleaned_fallback'
    )

    csv_filename = f"data/{base_cleaned_name}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Cleaned sheet saved as '{csv_filename}'.")

In [33]:
cleaned_dfs = {}

for sheet_name in sheet_names:
    print(f"\n--- Processing Sheet: {sheet_name} ---")

    df = _load_and_preprocess_headers(sheet_name)
    df = _drop_entirely_empty_rows(df, sheet_name)
    _initial_inspection(df)
    df = _handle_missing_values(df)
    df = _handle_duplicates(df)
    df = _standardize_datatypes(df)
    df = _address_categorical_inconsistencies(df)
    df = _outlier_inspection(df)
    df = _format_numerical_columns(df)

    cleaned_dfs[sheet_name] = df
    _save_cleaned_sheet(df, sheet_name)

print("\nAll sheets processed and cleaned.")


--- Processing Sheet: Seedlings measurements 1952024 ---

1. Initial Inspection (Head):
   Plant_NO_  Plant_height__(cm)  Shoot_length_(cm)  Root_length_(cm)  \
0          1                16.5                8.0               8.5   
1          2                15.0                7.0               8.0   
2          3                16.0                8.0               8.0   
3          4                12.5                6.0               6.5   
4          5                13.5                6.0               7.5   

  Head_diameter_(cm)  Stem_diameter__(cm)  Total_weight_(g)  Shoot_weight_(g)  \
0                9*5                  0.3                 6                 4   
1                8*5                  0.3                 5                 3   
2               10*9                  0.4                 6                 3   
3              6.5*6                  0.4                 5                 4   
4            6.5*6.5                  0.3                 4        